This notebook will take in a list of questions and calculate sentence similarity scores between each of them using the BERT model. We will begin by importing the dataset itself below. Credit for the "how-to" and code below goes to this [fantastic article](https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1). The main instruction is that the column with the text must be called 'text' for this script to work.

In [213]:
# Constants
dev = False
dataset = 'pubmed_abstracts' # https://www.kaggle.com/bonhart/pubmed-abstracts
column = 'deep_learning' # text
model_name = 'all-mpnet-base-v2' #'dmis-lab/biobert-base-cased-v1.2' #'multi-qa-mpnet-base-dot-v1' # all-mpnet-base-v2
cutoff = 0.5 # Note that cutoffs differ depending on the model used. 0.5 for all-mpnet-base-v2

In [214]:
import pandas as pd
dat = pd.read_csv('../data/processed/' + dataset + '.csv')

if dev == True:
    dat = dat.head(100)
    
display(dat.shape)
display(dat)

(13200, 17)

,Unnamed: 0,deep_learning,covid_19,human_connectome,virtual_reality,brain_machine_interfaces,electroactive_polymers,pedot_electrodes,neuroprosthetics,deep_learning_links,covid_19_links,human_connectome_links,virtual_reality_links,brain_machine_interfaces_links,electroactive_polymers_links,pedot_electrodes_links,neuroprosthetics_links
0,0,"(['Magnetic resonance spectroscopic imaging (MRSI) is a powerful molecular imaging modality but has very limited speed, resolution, and SNR tradeoffs. Construction of a low-dimensional model to effectively reduce the dimensionality of the imaging problem has recently shown great promise in improving these tradeoffs. This paper presents a new approach to model and reconstruct the spectroscopic signals by learning a nonlinear low-dimensional representation of the general MR spectra. Specifically, we trained a deep neural network to capture the low-dimensional manifold, where the high-dimensional spectroscopic signals reside. A regularization formulation is proposed to effectively integrate the learned model and physics-based data acquisition model for MRSI reconstruction with the capability to incorporate additional spatiospectral constraints. An efficient numerical algorithm was developed to solve the associated optimization problem involving back-propagating the trained network. Simulation and experimental results were obtained to demonstrate the representation power of the learned model and the ability of the proposed formulation in producing SNR-enhancing reconstruction from the practical MRSI data.'], 'Constrained Magnetic Resonance Spectroscopic Imaging by Learning Nonlinear Low-Dimensional Models.')","(['As cancer researchers shutter their labs to comply with COVID-19-related work restrictions, some are turning their attention, resources, and technical know-how to the challenge of tackling the deadly coronavirus.'], 'Cancer Labs Pivot to Battle COVID-19.')","(['For decades, it has been largely unknown to what extent multiple functional networks spatially overlap/interact with each other and jointly realize the total cortical function. Here, by developing novel sparse representation of whole-brain fMRI signals and by using the recently publicly released large-scale Human Connectome Project high-quality fMRI data, we show that a number of reproducible and robust functional networks, including both task-evoked and resting state networks, are simultaneously distributed in distant neuroanatomic areas and substantially spatially overlapping with each other, thus forming an initial collection of holistic atlases of functional networks and interactions (HAFNIs). More interestingly, the HAFNIs revealed two distinct patterns of highly overlapped regions and highly specialized regions and exhibited that these two patterns of areas are reciprocally localized, revealing a novel organizational principle of cortical function.'], 'Holistic atlases of functional networks and interactions reveal reciprocal organizational architecture of cortical function.')","(['To evaluate the differences between walking on an advanced robotic locomotion interface called the Treadport and walking overground with healthy subjects.'], 'The Treadport: Natural Gait on a Treadmill.')","(['All neural information systems (NIS) rely on sensing neural activity to supply commands and control signals for computers, machines and a variety of prosthetic devices. Invasive systems achieve a high signal-to-noise ratio (SNR) by eliminating the volume conduction problems caused by tissue and bone. An implantable brain machine interface (BMI) using intracortical electrodes provides excellent detection of a broad range of frequency oscillatory activities through the placement of a sensor in direct contact with cortex. This paper introduces a compact-sized implantable wireless 32-channel bidirectional brain machine interface (BBMI) to be used with freely-moving primates. The system is designed to monitor brain sensorimotor rhythms and presen

In [215]:
# Optional block where you filter and modify the dataset to fit your needs 
d = {column: dat[column], 'links': dat[column + '_links']}
dat = pd.DataFrame(data=d)
display(dat)

,deep_learning,links
0,"(['Magnetic resonance spectroscopic imaging (MRSI) is a powerful molecular imaging modality but has very limited speed, resolution, and SNR tradeoffs. Construction of a low-dimensional model to effectively reduce the dimensionality of the imaging problem has recently shown great promise in improving these tradeoffs. This paper presents a new approach to model and reconstruct the spectroscopic signals by learning a nonlinear low-dimensional representation of the general MR spectra. Specifically, we trained a deep neural network to capture the low-dimensional manifold, where the high-dimensional spectroscopic signals reside. A regularization formulation is proposed to effectively integrate the learned model and physics-based data acquisition model for MRSI reconstruction with the capability to incorporate additional spatiospectral constraints. An efficient numerical algorithm was developed to solve the associated optimization problem involving back-propagating the trained network. Simulation and experimental results were obtained to demonstrate the representation power of the learned model and the ability of the proposed formulation in producing SNR-enhancing reconstruction from the practical MRSI data.'], 'Constrained Magnetic Resonance Spectroscopic Imaging by Learning Nonlinear Low-Dimensional Models.')",https://www.ncbi.nlm.nih.gov/pubmed/31352337
1,"(['Existing deep convolutional neural networks (CNNs) have found major success in image deraining, but at the expense of an enormous number of parameters. This limits their potential applications, e.g., in mobile devices. In this paper, we propose a lightweight pyramid networt (LPNet) for single-image deraining. Instead of designing a complex network structure, we use domain-specific knowledge to simplify the learning process. In particular, we find that by introducing the mature Gaussian-Laplacian image pyramid decomposition technology to the neural network, the learning problem at each pyramid level is greatly simplified and can be handled by a relatively shallow network with few parameters. We adopt recursive and residual network structures to build the proposed LPNet, which has less than 8K parameters while still achieving the state-of-the-art performance on rain removal. We also discuss the potential value of LPNet for other low- and high-level vision tasks.'], 'Lightweight Pyramid Networks for Image Deraining.')",https://www.ncbi.nlm.nih.gov/pubmed/31329133
2,"(['Deep learning techniques have been increasingly used to provide more accurate and more accessible diagnosis of thorax diseases on chest radiographs. However, due to the lack of dense annotation of large-scale chest radiograph data, this computer-aided diagnosis task is intrinsically a weakly supervised learning problem and remains challenging. In this paper, we propose a novel deep convolutional neural network called Thorax-Net to diagnose 14 thorax diseases using chest radiography. Thorax-Net consists of a classification branch and an attention branch. The classification branch serves as a uniform feature extraction-classification network to free users from the troublesome hand-crafted feature extraction and classifier construction. The attention branch exploits the correlation between class labels and the locations of pathological abnormalities via analyzing the feature maps learned by the classification branch. Feeding a chest radiograph to the trained Thorax-Net, a diagnosis is obtained by averaging and binarizing the outputs of two branches. The proposed Thorax-Net model has been evaluated against three state-of-the-art deep learning models using the patientwise official split of the ChestX-ray14 dataset and against other five deep learning models using the imagewise random data split. Our results show that Thorax-Net achieves an average per-class area under the receiver operating characteristic curve (AUC) of 0.7876 and 0.896 in both experiments, respectively, which are higher than th

Now we will isolate the questions as an array of sentences, which will be fed into a pre-trained user-specified model. We note that there was a "module not found" error in the code below. The maintainer of the sentence-transformers package fixed it and requires the user to install via "pip install -U sentence-transformers." The code below takes a while to run, as the pre-trained model is quite large. A menu of models is [here](https://www.sbert.net/docs/pretrained_models.html)

In [216]:
from sentence_transformers import SentenceTransformer

sentences = dat[column].tolist()
model = SentenceTransformer(model_name)

sentence_embeddings = model.encode(sentences)
sentence_embeddings.shape

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg opt

Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg option direction
Ignored unknown kwarg opt

(13200, 768)

TODO Here, we make a topic model for our sentence similarity embedding. We will use the BERTopic library, [here](https://maartengr.github.io/BERTopic/index.html#quick-start)

From here, we're going to use cosine similarity to determine which questions are most similar to each other. One example of this is below. We compare the initial question to the first five questions after the initial question. We will display the questions and the similarity scores and see if it makes sense. 

In [217]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

cos_sim = cosine_similarity(
    [sentence_embeddings[20]],
    sentence_embeddings
).tolist()

df = pd.DataFrame({'text': sentences,
            'similarity': cos_sim[0]})

# Arrange by similarity
pd.set_option('display.max_colwidth', None)
display(df.sort_values(by = 'similarity', ascending = False))

,text,similarity
20,"(['Human genes often, through alternative splicing of pre-messenger RNAs, produce multiple mRNAs and protein isoforms that may have similar or completely different functions. Identification of splice sites is, therefore, crucial to understand the gene structure and variants of mRNA and protein isoforms produced by the primary RNA transcripts. Although many computational methods have been developed to detect the splice sites in humans, this is still substantially a challenging problem and further improvement of the computational model is still foreseeable. Accordingly, we developed DeepDSSR (deep donor splice site recognizer), a novel deep learning based architecture, for predicting human donor splice sites. The proposed method, built upon publicly available and highly imbalanced benchmark dataset, is comparable with the leading deep learning based methods for detecting human donor splice sites. Performance evaluation metrics show that DeepDSSR outperformed the existing deep learning based methods. Future work will improve the predictive capabilities of our model, and we will build a model for the prediction of acceptor splice sites.'], 'DeepDSSR: Deep Learning Structure for Human Donor Splice Sites Recognition.')",1.000000
8976,"(['The splicing of pre-mRNAs into mature transcripts is\xa0remarkable for its precision, but the mechanisms by which the cellular machinery achieves such specificity are incompletely understood. Here, we describe a deep neural network that accurately predicts splice junctions from an arbitrary pre-mRNA transcript sequence, enabling precise prediction of noncoding genetic variants that cause cryptic splicing. Synonymous and intronic mutations with predicted splice-altering consequence validate at a high rate on RNA-seq and are strongly deleterious in the human population. De novo mutations with predicted splice-altering consequence are significantly enriched in patients with autism and intellectual disability compared to healthy controls and validate against RNA-seq in 21 out of 28 of these patients. We estimate that 9%-11% of pathogenic mutations in patients with rare genetic disorders are caused by this previously underappreciated class of disease variation.'], 'Predicting Splicing from Primary Sequence with Deep Learning.')",0.817037
4821,"([""Experimental detection of RNA splicing branchpoints is difficult. To date, high-confidence experimental annotations exist for 18% of 3' splice sites in the human genome. We develop a deep-learning-based branchpoint predictor, LaBranchoR, which predicts a correct branchpoint for at least 75% of 3' splice sites genome-wide. Detailed analysis of cases in which our predicted branchpoint deviates from experimental data suggests a correct branchpoint is predicted in over 90% of cases. We use our predicted branchpoints to identify a novel sequence element upstream of branchpoints consistent with extended U2 snRNA base-pairing, show an association between weak branchpoints and alternative splicing, and explore the effects of genetic variants on branchpoints. We provide genome-wide branchpoint annotations and in silico mutagenesis scores at http://bejerano.stanford.edu/labranchor.""], 'A sequence-based, deep learning model accurately predicts RNA splicing branchpoints.')",0.811881
5341,"([""Alternative splice site selection is inherently competitive and the probability of a given splice site to be used also depends on the strength of neighboring sites. Here, we present a new model named the competitive splice site model (COSSMO), which explicitly accounts for these competitive effects and predicts the percent selected index (PSI) distribution over any number of putative splice sites. We model an alternative splicing event as the choice of a 3' acceptor site conditional on a fixed upstream 5' donor site or the choice of a 5' donor site conditional on a fixed 3' acceptor site. We build four different architectures that use convolutional layers, commun

We were going to run a UMAP on the vector space to get some intuition around what it looks like, but because UMAP has issues with python 3.9 at the moment, we're going to jump right to making a cosine similarity matrix that we will then turn into a graph. 

In [218]:
import sklearn

dist = sklearn.metrics.pairwise.cosine_distances(sentence_embeddings)
dist

# nn = sklearn.neighbors.kneighbors_graph(sentence_embeddings, n_neighbors = 1)
# nn

array([[0.        , 0.6991201 , 0.65333915, ..., 0.936666  , 0.97027946,
        0.9160899 ],
       [0.6991201 , 0.        , 0.64477223, ..., 0.9457232 , 0.9973163 ,
        0.92707026],
       [0.65333915, 0.64477223, 0.        , ..., 0.992597  , 1.0483567 ,
        1.0439858 ],
       ...,
       [0.936666  , 0.9457232 , 0.992597  , ..., 0.        , 0.7860487 ,
        0.50177336],
       [0.97027946, 0.9973163 , 1.0483567 , ..., 0.7860487 , 0.        ,
        0.5465266 ],
       [0.9160899 , 0.92707026, 1.0439858 , ..., 0.50177336, 0.5465266 ,
        0.        ]], dtype=float32)

In [219]:
import igraph as ig
import matplotlib.pyplot as plt

dist = np.where(dist == 0, 1, dist) # For the boolean below. Can't figure out compound boolean.

g = ig.Graph.Adjacency(dist < cutoff) # Need to convert to boolean
g = g.as_undirected()

if dat.shape[0] <= 1000:
    fig, ax = plt.subplots()
    ig.plot(g, target=ax)

Now we will do a quick measure of betweenness and eigenvector centrality to get a feel for what questions are the most central. We'll print the top 10 of each.

In [220]:
deg = g.degree()
btw = g.betweenness()
dat['degree'] = g.degree()
dat['betweenness'] = g.betweenness()
display(dat.sort_values(by='degree', ascending=False))
display(dat.sort_values(by='betweenness', ascending=False))

,deep_learning,links,degree,betweenness
6234,"(['Over the past decade, deep learning has demonstrated superior performances in solving many problems in various fields of medicine compared with other machine learning methods. To understand how deep learning has surpassed traditional machine learning techniques, in this review, we briefly explore the basic learning algorithms underlying deep learning. In addition, the procedures for building deep learning-based classifiers for seizure electroencephalograms and gastric tissue slides are described as examples to demonstrate the simplicity and effectiveness of deep learning applications. Finally, we review the clinical applications of deep learning in radiology, pathology, and drug discovery, where deep learning has been actively adopted. Considering the great advantages of deep learning techniques, deep learning will be increasingly and widely utilized in a wide variety of different areas in medicine in the coming decades.'], 'Applications of deep learning for the analysis of medical data.')",https://www.ncbi.nlm.nih.gov/pubmed/31140082,5253,688126.519952
668,"(['Deep learning is the state-of-the-art machine learning approach. The success of deep learning in many pattern recognition applications has brought excitement and high expectations that deep learning, or artificial intelligence (AI), can bring revolutionary changes in health care. Early studies of deep learning applied to lesion detection or classification have reported superior performance compared to those by conventional techniques or even better than radiologists in some tasks. The potential of applying deep-learning-based medical image analysis to computer-aided diagnosis (CAD), thus providing decision support to clinicians and improving the accuracy and efficiency of various diagnostic and treatment processes, has spurred new research and development efforts in CAD. Despite the optimism in this new era of machine learning, the development and implementation of CAD or AI tools in clinical practice face many challenges. In this chapter, we will discuss some of these issues and efforts needed to develop robust deep-learning-based CAD tools and integrate these tools into the clinical workflow, thereby advancing towards the goal of providing reliable intelligent aids for patient care.'], 'Deep Learning in Medical Image Analysis.')",https://www.ncbi.nlm.nih.gov/pubmed/32030660,4781,255366.371906
8090,"(['What has happened in machine learning lately, and what does it mean for the future of medical image analysis? Machine learning has witnessed a tremendous amount of attention over the last few years. The current boom started around 2009 when so-called deep artificial neural networks began outperforming other established models on a number of important benchmarks. Deep neural networks are now the state-of-the-art machine learning models across a variety of areas, from image analysis to natural language processing, and widely deployed in academia and industry. These developments have a huge potential for medical imaging technology, medical data analysis, medical diagnostics and healthcare in general, slowly being realized. We provide a short overview of recent advances and some associated challenges in machine learning applied to medical image processing and image analysis. As this has become a very broad and fast expanding field we will not survey the entire landscape of applications, but put particular focus on deep learning in MRI. Our aim is threefold: (i) give a brief introduction to deep learning with pointers to core references; (ii) indicate how deep learning has been applied to the entire MRI processing chain, from acquisition to image retrieval, from segmentation to disease prediction; (iii) provide a starting point for people interested in experimenting and perhaps contributing to the field of deep learning for medical imaging by pointing out good educational resources, state-of-the-art open-source code, and intere

,deep_learning,links,degree,betweenness
7533,"(['Being medical students, and having experienced different learning approaches ourselves, here, we discuss and critically analyse the importance of the deep learning approach that Chonkar et al. have presented, alongside emphasizing Case Based Learning, and their roles in life long medical learning.'], 'The association between deep learning approach and case based learning.')",https://www.ncbi.nlm.nih.gov/pubmed/30975134,617,2.393688e+06
6720,"(['Advances in biological and medical technologies have been providing us explosive volumes of biological and physiological data, such as medical images, electroencephalography, genomic and protein sequences. Learning from these data facilitates the understanding of human health and disease. Developed from artificial neural networks, deep learning-based algorithms show great promise in extracting features and learning patterns from complex data. The aim of this paper is to provide an overview of deep learning techniques and some of the state-of-the-art applications in the biomedical field. We first introduce the development of artificial neural network and deep learning. We then describe two main components of deep learning, i.e., deep learning architectures and model optimization. Subsequently, some examples are demonstrated for deep learning applications, including medical image classification, genomic sequence analysis, as well as protein structure classification and prediction. Finally, we offer our perspectives for the future directions in the field of deep learning.'], 'Deep Learning and Its Applications in Biomedicine.')",https://www.ncbi.nlm.nih.gov/pubmed/29522900,4413,7.799189e+05
5945,"(['Health care is evolving and with it the need to reform medical education. As the practice of medicine enters the age of artificial intelligence (AI), the use of data to improve clinical decision making will grow, pushing the need for skillful medicine-machine interaction. As the rate of medical knowledge grows, technologies such as AI are needed to enable health care professionals to effectively use this knowledge to practice medicine. Medical professionals need to be adequately trained in this new technology, its advantages to improve cost, quality, and access to health care, and its shortfalls such as transparency and liability. AI needs to be seamlessly integrated across different aspects of the curriculum. In this paper, we have addressed the state of medical education at present and have recommended a framework on how to evolve the medical education curriculum to include AI.'], 'Introducing Artificial Intelligence Training in Medical Education.')",https://www.ncbi.nlm.nih.gov/pubmed/31793895,569,6.886733e+05
6234,"(['Over the past decade, deep learning has demonstrated superior performances in solving many problems in various fields of medicine compared with other machine learning methods. To understand how deep learning has surpassed traditional machine learning techniques, in this review, we briefly explore the basic learning algorithms underlying deep learning. In addition, the procedures for building deep learning-based classifiers for seizure electroencephalograms and gastric tissue slides are described as examples to demonstrate the simplicity and effectiveness of deep learning applications. Finally, we review the clinical applications of deep learning in radiology, pathology, and drug discovery, where deep learning has been actively adopted. Considering the great advantages of deep learning techniques, deep learning will be increasingly and widely utilized in a wide variety of different areas in medicine in the coming decades.'], 'Applications of deep learning for the analysis of medical data.')",https://www.ncbi.nlm.nih.gov/pubmed/31140082,5253,6.881265e+05
9664,"(['The fast progress in research and development of multifunctional, distributed sensor networks has brought challenges in processing data from a large number of sensors. Us

From here, we run clustering to see if the the quesitons group into particular themes. We will use Louvain clustering, as it is often used in graph-based analysis.

In [221]:
clust = ig.Graph.community_multilevel(g)
#df = pd.DataFrame({'name': dat['Question'], 'cluster': clust.membership})
dat['cluster'] = clust.membership

# Get cluster ID, mine the per-cluster topics
for i in pd.unique(dat['cluster']):
    curr = dat[dat['cluster'] == i]
    if curr.shape[0] > 5:
        display(curr.sort_values(by='degree', ascending=False))
        

,deep_learning,links,degree,betweenness,cluster
8090,"(['What has happened in machine learning lately, and what does it mean for the future of medical image analysis? Machine learning has witnessed a tremendous amount of attention over the last few years. The current boom started around 2009 when so-called deep artificial neural networks began outperforming other established models on a number of important benchmarks. Deep neural networks are now the state-of-the-art machine learning models across a variety of areas, from image analysis to natural language processing, and widely deployed in academia and industry. These developments have a huge potential for medical imaging technology, medical data analysis, medical diagnostics and healthcare in general, slowly being realized. We provide a short overview of recent advances and some associated challenges in machine learning applied to medical image processing and image analysis. As this has become a very broad and fast expanding field we will not survey the entire landscape of applications, but put particular focus on deep learning in MRI. Our aim is threefold: (i) give a brief introduction to deep learning with pointers to core references; (ii) indicate how deep learning has been applied to the entire MRI processing chain, from acquisition to image retrieval, from segmentation to disease prediction; (iii) provide a starting point for people interested in experimenting and perhaps contributing to the field of deep learning for medical imaging by pointing out good educational resources, state-of-the-art open-source code, and interesting sources of data and problems related medical imaging.'], 'An overview of deep learning in medical imaging focusing on MRI.')",https://www.ncbi.nlm.nih.gov/pubmed/30553609,4555,242485.666499,0
12908,"(['Deep learning is an important new area of machine learning which encompasses a wide range of neural network architectures designed to complete various tasks. In the medical imaging domain, example tasks include organ segmentation, lesion detection, and tumor classification. The most popular network architecture for deep learning for images is the convolutional neural network (CNN). Whereas traditional machine learning requires determination and calculation of features from which the algorithm learns, deep learning approaches learn the important features as well as the proper weighting of those features to make predictions for new data. In this paper, we will describe some of the libraries and tools that are available to aid in the construction and efficient execution of deep learning as applied to medical images.'], 'Toolkits and Libraries for Deep Learning.')",https://www.ncbi.nlm.nih.gov/pubmed/28315069,4500,213632.463151,0
12848,"(['This review covers computer-assisted analysis of images in the field of medical imaging. Recent advances in machine learning, especially with regard to deep learning, are helping to identify, classify, and quantify patterns in medical images. At the core of these advances is the ability to exploit hierarchical feature representations learned solely from data, instead of features designed by hand according to domain-specific knowledge. Deep learning is rapidly becoming the state of the art, leading to enhanced performance in various medical applications. We introduce the fundamentals of deep learning methods and review their successes in image registration, detection of anatomical and cellular structures, tissue segmentation, computer-aided disease diagnosis and prognosis, and so on. We conclude by discussing research issues and suggesting future directions for further improvement.'], 'Deep Learning in Medical Image Analysis.')",https://www.ncbi.nlm.nih.gov/pubmed/28301734,4455,156034.225429,0
7197,"(['Deep learning is a class of machine learning methods that are gaining success and attracting interest in many domains, including computer vision, speech recognition, natural language processing, and playing games. Deep l

,deep_learning,links,degree,betweenness,cluster
9664,"(['The fast progress in research and development of multifunctional, distributed sensor networks has brought challenges in processing data from a large number of sensors. Using deep learning methods such as convolutional neural networks (CNN), it is possible to build smarter systems to forecasting future situations as well as precisely classify large amounts of data from sensors. Multi-sensor data from atmospheric pollutants measurements that involves five criteria, with the underlying analytic model unknown, need to be categorized, so do the Diabetic Retinopathy (DR) fundus images dataset. In this work, we created automatic classifiers based on a deep convolutional neural network (CNN) with two models, a simpler feedforward model with dual modules and an Inception Resnet v2 model, and various structural tweaks for classifying the data from the two tasks. For segregating multi-sensor data, we trained a deep CNN-based classifier on an image dataset extracted from the data by a novel image generating method. We created two deepened and one reductive feedforward network for DR phase classification. The validation accuracies and visualization results show that increasing deep CNN structure depth or kernels number in convolutional layers will not indefinitely improve the classification quality and that a more sophisticated model does not necessarily achieve higher performance when training datasets are quantitatively limited, while increasing training image resolution can induce higher classification accuracies for trained CNNs. The methodology aims at providing support for devising classification networks powering intelligent sensors.'], 'Study of the Application of Deep Convolutional Neural Networks (CNNs) in Processing Sensor Data and Biomedical Images.')",https://www.ncbi.nlm.nih.gov/pubmed/31426516,2952,631829.164154,1
4956,"(['Convolutional neural networks (CNNs) have been applied to visual tasks since the late 1980s. However, despite a few scattered applications, they were dormant until the mid-2000s when developments in computing power and the advent of large amounts of labeled data, supplemented by improved algorithms, contributed to their advancement and brought them to the forefront of a neural network renaissance that has seen rapid progression since 2012. In this review, which focuses on the application of CNNs to image classification tasks, we cover their development, from their predecessors up to recent state-of-the-art deep learning systems. Along the way, we analyze (1) their early successes, (2) their role in the deep learning renaissance, (3) selected symbolic works that have contributed to their recent popularity, and (4) several improvement attempts by reviewing contributions and challenges of over 300 publications. We also introduce some of their current trends and remaining challenges.'], 'Deep Convolutional Neural Networks for Image Classification: A Comprehensive Review.')",https://www.ncbi.nlm.nih.gov/pubmed/28599112,2381,437322.314148,1
6760,"(['Over the last years deep learning methods have been shown to outperform previous state-of-the-art machine learning techniques in several fields, with computer vision being one of the most prominent cases. This review paper provides a brief overview of some of the most significant deep learning schemes used in computer vision problems, that is, Convolutional Neural Networks, Deep Boltzmann Machines and Deep Belief Networks, and Stacked Denoising Autoencoders. A brief account of their history, structure, advantages, and limitations is given, followed by a description of their applications in various computer vision tasks, such as object detection, face recognition, action and activity recognition, and human pose estimation. Finally, a brief overview is given of future directions in designing deep learning schemes for computer vision problems and the challenges involved therein.'], 'Deep Learning for Computer Visi

,deep_learning,links,degree,betweenness,cluster
3822,"(['The rise of omics techniques has resulted in an explosion of molecular data in modern biomedical research. Together with information from medical images and clinical data, the field of omics has driven the implementation of personalized medicine. Biomedical and omics datasets are complex and heterogeneous, and extracting meaningful knowledge from this vast amount of information is by far the most important challenge for bioinformatics and machine learning researchers. In this context, there is an increasing interest in the potential of deep learning (DL) methods to create predictive models and to identify complex patterns from these large datasets. This chapter provides an overview of the main applications of DL methods in biomedical research, with focus on omics data analysis and precision medicine applications. DL algorithms and the most popular architectures are introduced first. This is followed by a review of some of the main applications and problems approached by DL in omics data and medical image analysis. Finally, implementations for improving the diagnosis, treatment, and classification of complex diseases are discussed.'], 'Deep Learning in Omics Data Analysis and Precision Medicine.')",https://www.ncbi.nlm.nih.gov/pubmed/31815397,3737,251759.850079,2
2103,"([""Deep Learning has boosted artificial intelligence over the past 5 years and is seen now as one of the major technological innovation areas, predicted to replace lots of repetitive, but complex tasks of human labor within the next decade. It is also expected to be 'game changing' for research activities in pharma and life sciences, where large sets of similar yet complex data samples are systematically analyzed. Deep learning is currently conquering formerly expert domains especially in areas requiring perception, previously not amenable to standard machine learning. A typical example is the automated analysis of images which are typically produced en-masse in many domains, e.\u2009g., in high-content screening or digital pathology. Deep learning enables to create competitive applications in so-far defined core domains of 'human intelligence'. Applications of artificial intelligence have been enabled in recent years by (i) the massive availability of data samples, collected in pharma driven drug programs (='big data') as well as (ii) deep learning algorithmic advancements and (iii) increase in compute power. Such applications are based on software frameworks with specific strengths and weaknesses. Here, we introduce typical applications and underlying frameworks for deep learning with a set of practical criteria for developing production ready solutions in life science and pharma research. Based on our own experience in successfully developing deep learning applications we provide suggestions and a baseline for selecting the most suited frameworks for a future-proof and cost-effective development.""], 'Developing Deep Learning Applications for Life Science and Pharma Industry.')",https://www.ncbi.nlm.nih.gov/pubmed/29341027,3717,431882.889859,2
4063,"(['In the era of big data, transformation of biomedical big data into valuable knowledge has been one of the most important challenges in bioinformatics. Deep learning has advanced rapidly since the early 2000s and now demonstrates state-of-the-art performance in various fields. Accordingly, application of deep learning in bioinformatics to gain insight from data has been emphasized in both academia and industry. Here, we review deep learning in bioinformatics, presenting examples of current research. To provide a useful and comprehensive perspective, we categorize research both by the bioinformatics domain (i.e. omics, biomedical imaging, biomedical signal processing) and deep learning architecture (i.e. deep neural networks, convolutional neural networks, recurrent neural networks, emergent architectures) and present brief descriptions of each study. Addit

,deep_learning,links,degree,betweenness,cluster
6234,"(['Over the past decade, deep learning has demonstrated superior performances in solving many problems in various fields of medicine compared with other machine learning methods. To understand how deep learning has surpassed traditional machine learning techniques, in this review, we briefly explore the basic learning algorithms underlying deep learning. In addition, the procedures for building deep learning-based classifiers for seizure electroencephalograms and gastric tissue slides are described as examples to demonstrate the simplicity and effectiveness of deep learning applications. Finally, we review the clinical applications of deep learning in radiology, pathology, and drug discovery, where deep learning has been actively adopted. Considering the great advantages of deep learning techniques, deep learning will be increasingly and widely utilized in a wide variety of different areas in medicine in the coming decades.'], 'Applications of deep learning for the analysis of medical data.')",https://www.ncbi.nlm.nih.gov/pubmed/31140082,5253,688126.519952,3
668,"(['Deep learning is the state-of-the-art machine learning approach. The success of deep learning in many pattern recognition applications has brought excitement and high expectations that deep learning, or artificial intelligence (AI), can bring revolutionary changes in health care. Early studies of deep learning applied to lesion detection or classification have reported superior performance compared to those by conventional techniques or even better than radiologists in some tasks. The potential of applying deep-learning-based medical image analysis to computer-aided diagnosis (CAD), thus providing decision support to clinicians and improving the accuracy and efficiency of various diagnostic and treatment processes, has spurred new research and development efforts in CAD. Despite the optimism in this new era of machine learning, the development and implementation of CAD or AI tools in clinical practice face many challenges. In this chapter, we will discuss some of these issues and efforts needed to develop robust deep-learning-based CAD tools and integrate these tools into the clinical workflow, thereby advancing towards the goal of providing reliable intelligent aids for patient care.'], 'Deep Learning in Medical Image Analysis.')",https://www.ncbi.nlm.nih.gov/pubmed/32030660,4781,255366.371906,3
6720,"(['Advances in biological and medical technologies have been providing us explosive volumes of biological and physiological data, such as medical images, electroencephalography, genomic and protein sequences. Learning from these data facilitates the understanding of human health and disease. Developed from artificial neural networks, deep learning-based algorithms show great promise in extracting features and learning patterns from complex data. The aim of this paper is to provide an overview of deep learning techniques and some of the state-of-the-art applications in the biomedical field. We first introduce the development of artificial neural network and deep learning. We then describe two main components of deep learning, i.e., deep learning architectures and model optimization. Subsequently, some examples are demonstrated for deep learning applications, including medical image classification, genomic sequence analysis, as well as protein structure classification and prediction. Finally, we offer our perspectives for the future directions in the field of deep learning.'], 'Deep Learning and Its Applications in Biomedicine.')",https://www.ncbi.nlm.nih.gov/pubmed/29522900,4413,779918.908620,3
5143,"(['Deep learning (DL) is affecting each and every sphere of public and private lives and becoming a tool for daily use. The power of DL lies in the fact that it tries to imitate the activities of neurons in the neocortex of human brain where the thought process takes place. Therefore, like the brain, it tries to learn 

,deep_learning,links,degree,betweenness,cluster
1117,"(['Artificial intelligence (AI) in medicine is a fast-growing field. The rise of deep learning algorithms, such as convolutional neural networks (CNNs), offers fascinating perspectives for the automation of medical image analysis. In this systematic review article, we screened the current literature and investigated the following question: ""Can deep learning algorithms for image recognition improve visual diagnosis in medicine?""'], 'Deep learning in medical image analysis: A third eye for doctors.')",https://www.ncbi.nlm.nih.gov/pubmed/31254638,3784,75166.300348,4
6877,"(['The implementation of clinical-decision support algorithms for medical imaging faces challenges with reliability and interpretability. Here, we establish a diagnostic tool based on a deep-learning framework for the screening of patients with common treatable blinding retinal diseases. Our framework utilizes transfer learning, which trains a neural network with a fraction of the data of conventional approaches. Applying this approach to a dataset of optical coherence tomography images, we demonstrate performance comparable to that of human experts in classifying age-related\xa0macular degeneration and diabetic macular\xa0edema. We also provide a more transparent and\xa0interpretable diagnosis by highlighting the regions recognized by the neural network. We further demonstrate the general applicability of our AI system for diagnosis of pediatric pneumonia\xa0using chest X-ray images. This tool may ultimately aid in expediting the diagnosis and referral\xa0of these treatable conditions, thereby facilitating earlier treatment, resulting in improved clinical outcomes. VIDEO ABSTRACT.'], 'Identifying Medical Diagnoses and Treatable Diseases by Image-Based Deep Learning.')",https://www.ncbi.nlm.nih.gov/pubmed/29474911,2995,27914.758538,4
10682,"(['Traditionally, medical discoveries are made by observing associations, making hypotheses from them and then designing and running experiments to test the hypotheses. However, with medical images, observing and quantifying associations can often be difficult because of the wide variety of features, patterns, colours, values and shapes that are present in real data. Here, we show that deep learning can extract new knowledge from retinal fundus images. Using deep-learning models trained on data from 284,335 patients and validated on two independent datasets of 12,026 and 999 patients, we predicted cardiovascular risk factors not previously thought to be present or quantifiable in retinal images, such as age (mean absolute error within 3.26 years), gender (area under the receiver operating characteristic curve (AUC)\u2009=\u20090.97), smoking status (AUC\u2009=\u20090.71), systolic blood pressure (mean absolute error within 11.23\u2009mmHg) and major adverse cardiac events (AUC\u2009=\u20090.70). We also show that the trained deep-learning models used anatomical features, such as the optic disc or blood vessels, to generate each prediction.'], 'Prediction of cardiovascular risk factors from retinal fundus photographs via deep learning.')",https://www.ncbi.nlm.nih.gov/pubmed/31015713,2850,38492.878233,4
3135,"(['Deep learning algorithms have been used to detect diabetic retinopathy (DR) with specialist-level accuracy. This study aims to validate one such algorithm on a large-scale clinical population, and compare the algorithm performance with that of human graders. A total of 25,326 gradable retinal images of patients with diabetes from the community-based, nationwide screening program of DR in Thailand were analyzed for DR severity and referable diabetic macular edema (DME). Grades adjudicated by a panel of international retinal specialists served as the reference standard. Relative to human graders, for detecting referable DR (moderate NPDR or worse), the deep learning algorithm had significantly higher sensitivity (0.97 vs. 0.74, p\u2009<\u20090.001), and a slightly l

,deep_learning,links,degree,betweenness,cluster
2726,"([""Deep brain stimulation (DBS) of the subthalamic nucleus (STN) has shown clinical potential for relieving the motor symptoms of advanced Parkinson's disease. While accurate localization of the STN is critical for consistent across-patients effective DBS, clear visualization of the STN under standard clinical MR protocols is still challenging. Therefore, intraoperative microelectrode recordings (MER) are incorporated to accurately localize the STN. However, MER require significant neurosurgical expertise and lengthen the surgery time. Recent advances in 7 T MR technology facilitate the ability to clearly visualize the STN. The vast majority of centers, however, still do not have 7 T MRI systems, and fewer have the ability to collect and analyze the data. This work introduces an automatic STN localization framework based on standard clinical MRIs without additional cost in the current DBS planning protocol. Our approach benefits from a large database of 7 T MRI and its clinical MRI pairs. We first model in the 7 T database, using efficient machine learning algorithms, the spatial and geometric dependency between the STN and its adjacent structures (predictors). Given a standard clinical MRI, our method automatically computes the predictors and uses the learned information to predict the patient-specific STN. We validate our proposed method on clinical T2 W MRI of 80 subjects, comparing with experts-segmented STNs from the corresponding 7 T MRI pairs. The experimental results show that our framework provides more accurate and robust patient-specific STN localization than using state-of-the-art atlases. We also demonstrate the clinical feasibility of the proposed technique assessing the post-operative electrode active contact locations.""], 'Automatic localization of the subthalamic nucleus on patient-specific clinical MRI by incorporating 7 T MRI and machine learning: Application in deep brain stimulation.')",https://www.ncbi.nlm.nih.gov/pubmed/30379376,645,144388.873630,5
290,"(['Deep brain stimulation (DBS) is an established and effective treatment for several movement disorders and is being developed to treat a host of neuropsychiatric disorders including epilepsy, chronic pain, obsessive compulsive disorder, and depression. However, the neural mechanisms through which DBS produces therapeutic benefits, and in some cases unwanted side effects, in these disorders are only partially understood. Non-invasive neuroimaging techniques that can assess the neural effects of active stimulation are important for advancing our understanding of the neural basis of DBS therapy. Magnetoencephalography (MEG) is a safe, passive imaging modality with relatively high spatiotemporal resolution, which makes it a potentially powerful method for examining the cortical network effects of DBS. However, the degree to which magnetic artifacts produced by stimulation and the associated hardware can be suppressed from MEG data, and the comparability between signals measured during DBS-on and DBS-off conditions, have not been fully quantified. The present study used machine learning methods in conjunction with a visual perception task, which should be relatively unaffected by DBS, to quantify how well neural data can be salvaged from artifact contamination introduced by DBS and how comparable DBS-on and DBS-off data are after artifact removal. Machine learning also allowed us to determine whether the spatiotemporal pattern of neural activity recorded during stimulation are comparable to those recorded when stimulation is off. The spatiotemporal patterns of visually evoked neural fields could be accurately classified in all 8 patients with DBS implants during both DBS-on and DBS-off conditions and performed comparably across those two conditions. Further, the classification accuracy for classifiers trained on the spatiotemporal patterns evoked during DBS-on trials and applied to DBS-off trials, and v

,deep_learning,links,degree,betweenness,cluster
7533,"(['Being medical students, and having experienced different learning approaches ourselves, here, we discuss and critically analyse the importance of the deep learning approach that Chonkar et al. have presented, alongside emphasizing Case Based Learning, and their roles in life long medical learning.'], 'The association between deep learning approach and case based learning.')",https://www.ncbi.nlm.nih.gov/pubmed/30975134,617,2.393688e+06,6
11606,"(['Students take three approaches to learning and studying: deep, surface and strategic, influenced by the learning environment. Following the General Medical Council\'s report ""Tomorrow\'s Doctors,"" a deep approach was cultivated in Years 1 and 2 of a university undergraduate medical programme by introducing explicit written learning objectives constructed according to Biggs\' SOLO taxonomy, problem-based learning and constructively aligned in-course assignments and examinations. The effect of these changes was measured with the Approaches to Study Skills Inventory for Students (ASSIST). Scores were highest for a deep approach and lowest for a surface approach and showed relatively little change during the degree programme, apart from a slight fall in the scores for a surface approach, particularly for students undertaking an intercalated science degree. Possible explanations include: students\' approaches may be established prior to university entry; deep scores were already high at the beginning of the programme and may be difficult to increase further; the changes in learning environment may not be strong enough to alter approaches which students perceive as having been successful.'], ""Medical students' approaches to learning over a full degree programme."")",https://www.ncbi.nlm.nih.gov/pubmed/22927717,421,2.227699e+04,6
6230,"([""Students use three approaches to learning and studying: deep, surface and strategic. These are influenced by the learning environment. In response to the General Medical Council's report 'Tomorrow's Doctors', the second year of the medical course at the University of Edinburgh was changed to promote deep learning, with learning objectives constructed according to the SOLO taxonomy, learning methods such as problem-based learning and constructively aligned written assignments and examinations. The Approaches to Study Skills Inventory for Students (ASSIST) was used to evaluate the effect of these changes. Scores were highest for deep approaches and lowest for surface approaches and showed almost no change during the course. There are various possible explanations. The students already scored highly on deep approaches at the beginning of Year 2 and it may be difficult to increase the deep scores further, particularly over the relatively short period of the study. Alternatively, the effect of the changes in learning environment may not be strong enough to change entrenched approaches which have hitherto been successful.""], ""Can we influence medical students' approaches to learning?"")",https://www.ncbi.nlm.nih.gov/pubmed/16147792,410,1.598396e+04,6
12355,"(['If professionals are to be equipped better to meet the needs of modern health care systems and the standards of practice required, significant educational change is still required. Educational change requires leadership, and lack of educational leadership may have impeded change in the past. In practical terms standards refer to outcomes, and thus an outcome based approach to clinical education is advocated as the one most likely to provide an appropriate framework for organisational and system change. The provision of explicit statements of learning intent, an educational process enabling acquisition and demonstration of these, and criteria for ensuring their achievement are the key features of such a framework. The derivation of an appropriate outcome set should emphasise what the learners will be able to do following the learning experience,

,deep_learning,links,degree,betweenness,cluster
6800,"([""The future of human life in the world's river deltas depends on the success of water management. To deal with uncertainties about the future, policymakers in the Netherlands have used scenarios to develop water management strategies for the coastal zone of the Rhine-Meuse delta. In this paper we reflect on six decades of scenario use in the Netherlands, and provide recommendations for future studies. Based on two criteria, 'Decision robustness' and 'Learning success', we conclude that (1) the possibilities for robust decisionmaking increased through a paradigm shift from predicting to exploring futures, but the scenario method is not yet fully exploited for decisionmaking under uncertainty; and (2) the scenarios enabled learning about possible impacts of developments and effectiveness of policy options. New scenario approaches are emerging to deal with the deep uncertainties water managers are currently facing.""], 'A history of futures: A review of scenario use in water policy studies in the Netherlands.')",https://www.ncbi.nlm.nih.gov/pubmed/23471143,4,2.5,31
11849,"([""Policy-making is usually about risk management. Thus, the handling of uncertainty in science is central to its support of sound policy-making. There is value in scientists engaging in a deep conversation with policy-makers and others, not merely 'delivering' results or analyses and then playing no further role. Communicating the policy relevance of different varieties of uncertainty, including imprecision, ambiguity, intractability and indeterminism, is an important part of this conversation. Uncertainty is handled better when scientists engage with policy-makers. Climate policy aims both to alter future risks (particularly via mitigation) and to take account of and respond to relevant remaining risks (via adaptation) in the complex causal chain that begins and ends with individuals. Policy-making profits from learning how to shift the distribution of risks towards less dangerous impacts, even if the probability of events remains uncertain. Immediate value lies not only in communicating how risks may change with time and how those risks may be changed by action, but also in projecting how our understanding of those risks may improve with time (via science) and how our ability to influence them may advance (via technology and policy design). Guidance on the most urgent places to gather information and realistic estimates of when to expect more informative answers is of immediate value, as are plausible estimates of the risk of delaying action. Risk assessment requires grappling with probability and ambiguity (uncertainty in the Knightian sense) and assessing the ethical, logical, philosophical and economic underpinnings of whether a target of '50 per cent chance of remaining under +2(°)C' is either 'right' or 'safe'. How do we better stimulate advances in the difficult analytical and philosophical questions while maintaining foundational scientific work advancing our understanding of the phenomena? And provide immediate help with decisions that must be made now?""], 'Uncertainty in science and its role in climate policy.')",https://www.ncbi.nlm.nih.gov/pubmed/22042899,4,4.5,31
11255,"(['In this paper, we discuss the multiple dimensions of water security and define a set of thematic challenges for science, policy and governance, based around cross-scale dynamics, complexity and uncertainty. A case study of the Saskatchewan River basin (SRB) in western Canada is presented, which encompasses many of the water-security challenges faced worldwide. A science agenda is defined based on the development of the SRB as a large-scale observatory to develop the underpinning science and social science needed to improve our understanding of water futures under societal and environmental change. We argue that non-stationarity poses profound challenges for existing science and that new integration of the natural science

Now, we're going to pull the graph out as an edgelist and get a feel for who is connected to who. We will then export the edgelist for import into Neo4J. 

TODO 1) make this a searchable umap. 2) place this into neo4j 3) set up the virtual environemnt and stop installing anything to the regular environemnt

In [222]:
el = g.get_edge_dataframe()
el

,source,target
edge ID,,
0,2,8
1,4,10
2,9,10
3,2,11
4,3,11
...,...,...
4538807,13169,13199
4538808,13179,13199
4538809,13184,13199


Now we have to convert the edges from their edge IDs to the questions that are in the order of the IDs. The reason we're seeing numbers right now is that the original adjacency matrix was made with the IDs and not the questions themselves. We're going to do that using a simple conversion function. We're going to do this by creating a dictionary. 

In [223]:
q_dict = {}

for i in range(0, len(sentences)):
    q_dict[i] = sentences[i]


In [224]:
e1 = [q_dict[i] for i in el['source']]
e2 = [q_dict[i] for i in el['target']]

el_df = pd.DataFrame({'edge1': e1, 'edge2': e2})
el_df 



,edge1,edge2
0,"(['Deep learning techniques have been increasingly used to provide more accurate and more accessible diagnosis of thorax diseases on chest radiographs. However, due to the lack of dense annotation of large-scale chest radiograph data, this computer-aided diagnosis task is intrinsically a weakly supervised learning problem and remains challenging. In this paper, we propose a novel deep convolutional neural network called Thorax-Net to diagnose 14 thorax diseases using chest radiography. Thorax-Net consists of a classification branch and an attention branch. The classification branch serves as a uniform feature extraction-classification network to free users from the troublesome hand-crafted feature extraction and classifier construction. The attention branch exploits the correlation between class labels and the locations of pathological abnormalities via analyzing the feature maps learned by the classification branch. Feeding a chest radiograph to the trained Thorax-Net, a diagnosis is obtained by averaging and binarizing the outputs of two branches. The proposed Thorax-Net model has been evaluated against three state-of-the-art deep learning models using the patientwise official split of the ChestX-ray14 dataset and against other five deep learning models using the imagewise random data split. Our results show that Thorax-Net achieves an average per-class area under the receiver operating characteristic curve (AUC) of 0.7876 and 0.896 in both experiments, respectively, which are higher than the AUC values obtained by other deep models when they were all trained with no external data.'], 'Thorax-Net: An Attention Regularized Deep Neural Network for Classification of Thoracic Diseases on Chest Radiography.')","(['To assess the use of deep learning (DL) for computer-assisted glaucoma identification, and the impact of training using images selected by an active learning strategy, which minimizes labelling cost. Additionally, this study focuses on the explainability of the glaucoma classifier.'], 'Accurate prediction of glaucoma from colour fundus images with a convolutional neural network that relies on active and transfer learning.')"
1,"(['The most common applications of artificial intelligence (AI) in drug treatment have to do with matching patients to their optimal drug or combination of drugs, predicting drug-target or drug-drug interactions, and optimizing treatment protocols. This review outlines some of the recently developed AI methods aiding the drug treatment and administration process. Selection of the best drug(s) for a patient typically requires the integration of patient data, such as genetics or proteomics, with drug data, like compound chemical descriptors, to score the therapeutic efficacy of drugs. The prediction of drug interactions often relies on similarity metrics, assuming that drugs with similar structures or targets will have comparable behavior or may interfere with each other. Optimizing the dosage schedule for administration of drugs is performed using mathematical models to interpret pharmacokinetic and pharmacodynamic data. The recently developed and powerful models for each of these tasks are addressed, explained, and analyzed here.'], 'Artificial Intelligence in Drug Treatment.')","(['Membrane proteins, the most important drug targets, account for around 30% of total proteins encoded by the genome of living organisms. An important role of these proteins is to bind adenosine triphosphate (ATP), facilitating crucial biological processes such as metabolism and cell signaling. There are several reports elucidating ATP-binding sites within proteins. However, such studies on membrane proteins are limited. Our prediction tool, DeepATP, combines evolutionary information in the form of Position Specific Scoring Matrix and two-dimensional Convolutional Neural Network to predict ATP-binding sites in membrane proteins with an MCC of 0.89 and an AUC of 99%. Compared to recently published ATP-bin

In [ ]:
el_df.to_csv('../output/' + dataset + '_' + model_name + '_edgelist_dist_' + str(cutoff) + '.csv', encoding='utf-8-sig')
dat.to_csv('../output/' + dataset + '_' + model_name + '_analyzed_' + str(cutoff) + '.csv', encoding='utf-8-sig')
#model_out.to_csv('../output/' + dataset + '_topic_model' + str(cutoff) + '.csv', encoding='utf-8-sig')